# Library

## install

command에서 아래 설치 (anaconda prompt가 아님)

```cmd
pip install nni
```

# Hyperparameter Tuning

모델의 성능을 최적화하기 위해 하이퍼파라미터를 조정 <br>
하이퍼파라미터는 모델 구조나 학습 과정에 영향을 미치는 설정 값으로, 예를 들어 학습률, 배치 크기 hidden dim의 크기 등이 존재. <br>
튜닝을 통해 최적의 하이퍼파라미터 조합을 찾아 모델의 예측 성능 개선을 꾀함. <br>
딥러닝에서는 sklearn에서 사용하였던 tuning 기법 대신 NNI 사용 <br>

## NNI (Neural Network Intelligence)

github: https://github.com/microsoft/nni <br>
documentation: https://nni.readthedocs.io/en/stable/index.html <br>

<br>

NNI는 마이크로소프트에서 개발한 오픈소스 자동화된 머신러닝 도구. <br>
주로 **하이퍼파라미터 튜닝**, 모델 탐색, AutoML 작업을 지원. <br>
NNI는 베이지안 최적화 등의 다양한 튜닝 알고리즘을 사용해 최적의 하이퍼파라미터를 찾고, 실험 결과를 시각화할 수 있는 기능도 제공. <br>
&nbsp;&nbsp;&nbsp;&nbsp;-> 연구자와 개발자가 모델 성능을 쉽게 향상시킬 수 있도록 도움 <br>

## 사용 방법

cmd에서 pytorch가 설치된 환경을 사용하기 위해 아래 command 입력

```cmd
conda init
```

<br>

이후 pytorch가 설치된 환경으로 변경

```cmd
conda activate [env_name]
```

<br>

nni 설치 이후 command에서 아래의 명령어 입력

```cmd
nnictl create --config config.yaml --port 8080
nnictl stop -a           # 모든 실험 종료
nnictl stop              # 모든 실험 종료
nnictl stop exp_id       # 특정 실험 종료
```

- config.yaml: search area, nni option을 설정하는 yaml file
- port: 웹통신을 위한 것으로 한 실험에 하나의 port가 매칭


## config

NNI을 실행하기 위한 환경 설정으로 아래의 내용을 설정

> ```python
> searchSpace:                    # parameter가 갖는 값의 범위를 지정
>   batch_size:
>     _type: choice               # 아래의 값 중에서 선택
>     _value: [16, 32, 64]
>   hidden_dim:
>     _type: randint              # 아래의 범위 중에서 선택
>     _value: [32, 512]
>   learning_rate:
>     _type: uniform              # 아래의 a, b를 갖는 uniform 분포에서 값 선택
>     _value: [0.001, 0.2]
>   dropout_ratio:
>     _type: uniform
>     _value: [0.01, 0.5]
> 
> 
> trialCommand: python train.py   # 코드를 실행할 때 사용할 command
> trialCodeDirectory: .           # 코드가 실행될 directory
> 
> trial_concurrency:              # 동시에 실험할 횟수
> max_trial_number: 100           # 최대 실험 횟수
> 
> tuner:
>   name: TPE                     # SMBO 접근 방법으로, 과거 측정을 기반으로 하이퍼파라미터의 성능을 근사화하기 위해
>                                 # 모델을 순차적으로 구성한 다음 이 모델을 기반으로 테스트할 새로운 하이퍼파라미터를 선택
>   classArgs:
>     optimize_mode: minimize     # nni.report_final_result를 어떤 방향으로 최적화 할지 결정
> 
> trainingService:
>   platform: local       # NNI를 실행할 환경 설정.
>                         # local: 현재 사용 중인 PC
>                         # remote: server
> 
> debug: true             # debug 실행 유무
> ```

## NNI parameter

nni.get_next_parameter()를 통해 실험에 사용할 parameter를 전달받음. <br>
전달받은 parameter를 모델에 넘겨줘야 함. <br>
config를 사용하여 모델에 넘기는 것이 사용하기 용이. <br>
train.py에서 아래와 같이 내용 추가

<br>

<font style="font-size:20px"> 사용 방법 </font> <p>

> ```python
> if configs.get('nni'):
>     params = nni.get_next_parameter()     # params: dict
>     configs.update({'batch_size': params.get('batch_size')})
>     configs.update({'hidden_dim': params.get('hidden_dim')})
>     configs.update({'learning_rate': params.get('learning_rate')})
>     configs.update({'dropout_ratio': params.get('dropout_ratio')})
> ```

<br>

코드 중간에 아래와 같이 평가를 위한 metric을 넘겨줘야 함

> ```python
> nni.report_intermediate_result([metric])  # 학습 중간 과정을 tracking하기 위한 metric
> nni.report_final_result([metric])         # 최종 결과를 위한 metric
> ```